In [34]:
# <img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [36]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy        : 1.16.5


In [37]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# !pip freeze > requirements.txt

In [38]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [39]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [40]:
VERSION    = 16
DIR  = './Project_7_data/' # подключил к ноутбуку внешний датасет
VAL_SIZE   = 0.20   # 20%

marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

# Data

In [41]:
# !ls '../input'

In [42]:
train = pd.read_csv(DIR+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR+'test.csv')
sample_submission = pd.read_csv(DIR+'sample_submission.csv')

In [43]:
train_2 = pd.read_csv(DIR+'all_auto_ru_14_11_2020.csv') # датасет для обучения модели

In [44]:
# train.head(5)

In [45]:
# train.info()

In [46]:
# test.head(5)

In [47]:
# test.info()

## Data Preprocessing

In [48]:
# ... 

In [49]:
train.dropna(subset=['productionDate','mileage'], inplace=True)
train.dropna(subset=['price'], inplace=True)

train_2.dropna(subset=['productionDate','mileage'], inplace=True)
train_2.dropna(subset=['price'], inplace=True)

In [50]:
# для baseline просто возьму пару схожих признаков без полной обработки
columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']
df_train = train[columns]
df_test = test[columns]

df_train_2 = train_2[columns]

In [51]:
y = train['price']

y_2 = train_2['price']

## Label Encoding

In [52]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

df_train_2['sample'] = 1 # помечаем где у нас трейн
data_2 = df_test.append(df_train_2, sort=False).reset_index(drop=True) # объединяем

C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [53]:
for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data[colum] = data[colum].astype('category').cat.codes
    
for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data_2[colum] = data_2[colum].astype('category').cat.codes

In [54]:
data.head()

,bodyType,brand,productionDate,engineDisplacement,mileage,sample
0,176,29,2014,11,74000,0
1,176,29,2017,27,60563,0
2,176,29,2014,33,88000,0
3,176,29,2014,27,95000,0
4,176,29,2012,33,58536,0


In [55]:
data_2.head()

,bodyType,brand,productionDate,engineDisplacement,mileage,sample
0,174,29,2014,10,74000,0
1,174,29,2017,21,60563,0
2,174,29,2014,27,88000,0
3,174,29,2014,21,95000,0
4,174,29,2012,27,58536,0


In [56]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

X_2 = data_2.query('sample == 1').drop(['sample'], axis=1)
X_sub_2 = data_2.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




tmp_train = X_train.copy()
tmp_train['price'] = y_train

# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [58]:
%%time
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

Wall time: 29.9 s


In [59]:
%%time
model_2 = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model_2.fit(X_train_2, y_train_2,
         #cat_features=cat_features_ids,
         eval_set=(X_test_2, y_test_2),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model_2.save_model('catboost_single_model_baseline.model')

Wall time: 26.7 s


In [60]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

predict_2 = model_2.predict(X_test_2)
print(f"Точность модели_2 по метрике MAPE: {(mape(y_test_2, predict_2))*100:0.2f}%")

Точность модели по метрике MAPE: 18.11%
Точность модели_2 по метрике MAPE: 17.83%


Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [61]:
np.log(y_train)

53199    13.060488
75086    14.762874
43113    12.762827
69668    12.785491
70499    11.918391
           ...    
6291     14.508657
55136    12.468437
77178    13.217674
865      12.641097
15861    14.403297
Name: price, Length: 71174, dtype: float64

In [62]:
%%time
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

Wall time: 28.5 s


In [63]:
%%time
model_2 = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model_2.fit(X_train_2, np.log(y_train_2),
         #cat_features=cat_features_ids,
         eval_set=(X_test_2, np.log(y_test_2)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model_2.save_model('catboost_single_model_2_2_baseline.model')

Wall time: 25.8 s


In [64]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

predict_test_2 = np.exp(model_2.predict(X_test_2))
predict_submission_2 = np.exp(model_2.predict(X_sub_2))

In [68]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 15.07%


In [65]:
print(f"Точность модели_2 по метрике MAPE: {(mape(y_test_2, predict_test_2))*100:0.2f}%")

Точность модели_2 по метрике MAPE: 15.20%


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [66]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_1.csv', index=False)
sample_submission.head()

,sell_id,price
0,1100575026,479298.120078
1,1100549428,753612.285336
2,1100658222,679251.364011
3,1100937408,579601.707211
4,1101037972,618111.240425


In [67]:
sample_submission['price'] = predict_submission_2
sample_submission.to_csv(f'submission_2_2.csv', index=False)
sample_submission.head()

,sell_id,price
0,1100575026,371150.191920
1,1100549428,652101.371590
2,1100658222,612710.351001
3,1100937408,474914.573102
4,1101037972,503648.853131


В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

# ===================== EX ==========================

In [23]:
train[train.brand.apply(lambda x: x in marks_for_parsing)].shape

(49309, 26)